In [145]:
import random

In [146]:
class Player:
    def __init__(self, 
               prob_win_serve = 0.5, 
               prob_win_return = 0.5,
               prob_win_important_point = 0.5):
        '''
            Player Class - Represents a tennis player.
                prob_win_serve            :: float between 0 and 1
                prob_win_return           :: float between 0 and 1
                prob_win_important_point  :: float between 0 and 1
        '''
        self.p_serve = prob_win_serve
        self.p_return = prob_win_return
        self.p_important = prob_win_important_point
           
    def PlayPoint(self, isServe, gameScore, setScore):
        '''
            Simulates a point in a tennis match.
                isServe   :: boolean
                gameScore :: list [A,B] 
                setScore  :: list [A,B]
            return 0 (lose) or 1 (win)
        '''
        
        p = random.uniform(0,1)
        if isServe:
            return self.p_serve > p
        else:
            return self.p_return > p

In [207]:
class TennisMatch:
    tiebreak = [6,6]
    scoreMap = {
        0  : '15',
        '15': '30',
        '30': '40',
        '40': 'W',
        'A' : 'W'
    }
    
    def __init__(self, Player, bestOfFive = False, serveFirst = True):
        self.Player = Player  # Represents Player A
        self.currentGame = [0,0]
        self.currentSet = [0,0]
        self.setCount = [0,0]
        self.currentGameHistory = []
        self.currentSetHistory = []
        self.gameHistory = []
        self.setHistory = []
        self.isServe = serveFirst
        self.isFinished = False
        self.winningSetNumber = 3 if bestOfFive else 2
        self.winner = None
        
        self.pointsWon = [0,0]
        
    def PlayMatch(self):
        '''
            Simulates playing a tennis match.
        '''
        while True:
            if self.IsMatchFinished():
                break
            else:
                self.PlaySet()
        self.ShowResult()
            
    def PlaySet(self):
        '''
            Simulates a tennis set.
        '''
        while True:
            result = self.IsSetFinished()
            if result:
                self.SetFinished(result)
                break
            elif self.IsTiebreak():
                self.PlayTiebreak()
            else:
                self.PlayGame()
                
    
    def PlayGame(self):
        '''
            Simulates a tennis game.
        '''
        while True:
            result = self.IsGameFinished()
            if result:
                self.GameFinished(result)
                break
            else:
                playerAWins = self.Player.PlayPoint(self.isServe, self.currentGame, self.currentSet)
                if playerAWins:
                    self.UpdateGameScore(False)
                else:
                    self.UpdateGameScore(True)
        return
    
    def PlayTiebreak(self):
        '''
            Simulate a tiebreak.
        '''
        while True:
            result = self.IsTiebreakFinished()
            if result:
                self.GameFinished(result)
                break
            else:
                playerAWins = self.Player.PlayPoint(self.isServe, self.currentGame, self.currentSet)
                if playerAWins:
                    self.UpdateTiebreakScore(False)
                else:
                    self.UpdateTiebreakScore(True)
    
    def IsMatchFinished(self):
        '''
            Determines whether the match has finished.
        '''
        if self.setCount[0] == self.winningSetNumber:
            self.winner = 'A'
            return True
        elif self.setCount[1] == self.winningSetNumber:
            self.winner = 'B'
            return True
        return False
    
    def IsSetFinished(self):
        '''
            Determines whether the set has finished and who won.
            returns 'A', 'B', or False
        '''
        if (abs(self.currentSet[0] - self.currentSet[1]) > 1 and max(self.currentSet) > 5) or sum(self.currentSet) == 13:
            self.setHistory.append(self.currentSet)
            if (self.currentSet[0] > self.currentSet[1]):
                return 'A'
            else:
                return 'B'
        return False
    
    def IsGameFinished(self):
        '''
            Determines whether the game has finished and who won.
            returns 'A', 'B', or False
        '''
        if self.currentGame[0] == 'W':
            return 'A'
        elif self.currentGame[1] == 'W':
            return 'B'
        else:
            return False
        
    def IsTiebreakFinished(self):
        '''
            Determines whether the tiebreak has finished and who won.
            returns 'A', 'B', or False
        '''
        if abs(self.currentGame[0] - self.currentGame[1]) > 1 and max(self.currentGame) > 6:
            if self.currentGame[0] > self.currentGame[1]:
                return 'A'
            else:
                return 'B'
        return False
    
    def IsTiebreak(self):
        if self.currentSet == tiebreak:
            return True
        return False
        
    def UpdateGameScore(self, isPlayerBWin):
        '''
            Updates score of current game.
        '''
        self.pointsWon[isPlayerBWin] += 1
        
        self.currentGameHistory.append(self.currentGame)
        if self.currentGame[not isPlayerBWin] == 'A':
            self.currentGame[not isPlayerBWin] = '40'
        elif self.currentGame == ['40', '40']:
            self.currentGame[isPlayerBWin] = 'A'
        else:
            self.currentGame[isPlayerBWin] = scoreMap[self.currentGame[isPlayerBWin]]
            
    def UpdateTiebreakScore(self, isPlayerBWin):
        '''
            Updates score of current tiebreak.
        '''
        self.pointsWon[isPlayerBWin] += 1
        
        self.currentGameHistory.append(self.currentGame)
        self.currentGame[isPlayerBWin] += 1
        
    def SetFinished(self, winner):
        self.currentSetHistory.append(self.currentSet)
        self.currentSet = [0,0]
        
        if winner == 'A':
            self.setCount[0] += 1
        else:
            self.setCount[1] += 1
    
    def GameFinished(self, winner):
        self.currentGameHistory.append(self.currentGame)
        self.currentGame = [0,0]
        self.isServe = not self.isServe
        
        if winner == 'A':
            self.currentSet[0] += 1
        else:
            self.currentSet[1] += 1        
    
    def Reset(self, bestOfFive = False, serveFirst = True):
        self.currentGame = [0,0]
        self.currentSet = [0,0]
        self.setCount = [0,0]
        self.currentGameHistory = []
        self.currentSetHistory = []
        self.gameHistory = []
        self.setHistory = []
        self.isServe = serveFirst
        self.isFinished = False
        self.winningSetNumber = 3 if bestOfFive else 2
        self.winner = None
        
        
    def ShowResult(self):
        print('Points Won', self.pointsWon)
        print('Set Count', self.setCount)
        print('Set History', self.setHistory)
        print('Winner', self.winner)
        

In [208]:
for i in range(10):
    PlayerA = Player()
    Match = TennisMatch(PlayerA, bestOfFive=True)
    Match.PlayMatch()


Points Won [144, 156]
Set Count [2, 3]
Set History [[6, 4], [6, 4], [1, 6], [4, 6], [3, 6]]
Winner B
Points Won [137, 118]
Set Count [3, 1]
Set History [[3, 6], [6, 2], [6, 3], [7, 5]]
Winner A
Points Won [126, 143]
Set Count [2, 3]
Set History [[4, 6], [1, 6], [6, 4], [6, 2], [2, 6]]
Winner B
Points Won [159, 158]
Set Count [2, 3]
Set History [[6, 2], [3, 6], [5, 7], [6, 3], [4, 6]]
Winner B
Points Won [110, 89]
Set Count [3, 0]
Set History [[7, 6], [6, 2], [6, 2]]
Winner A
Points Won [161, 172]
Set Count [2, 3]
Set History [[6, 2], [7, 5], [3, 6], [1, 6], [2, 6]]
Winner B
Points Won [82, 107]
Set Count [0, 3]
Set History [[3, 6], [6, 7], [3, 6]]
Winner B
Points Won [125, 106]
Set Count [3, 1]
Set History [[6, 1], [6, 3], [3, 6], [6, 3]]
Winner A
Points Won [183, 173]
Set Count [3, 2]
Set History [[6, 2], [4, 6], [4, 6], [6, 2], [7, 6]]
Winner A
Points Won [109, 90]
Set Count [3, 0]
Set History [[7, 6], [6, 3], [6, 1]]
Winner A


In [ ]:
'''
    Possible States
    - Close to losing = increase probability
    - Consistency 
    - Nervousness
    - Performs better or worse depending on game/set score
    
'''